In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.insert(0, "..")

from data_split.data_utils import DataSplitter

In [2]:
# spectra_matrix = np.load("../data/DRIAMS-A/spectra_binned_6000_all.npy")
# spectra_matrix.shape

In [3]:
driams_long_table = pd.read_csv("../processed_data/DRIAMS_combined_long_table.csv")
driams_long_table
dsplit = DataSplitter(driams_long_table, dataset="B")
samples_list = sorted(dsplit.long_table["sample_id"].unique())
drugs_set = sorted(dsplit.long_table["drug"].unique())
print(samples_list[:4])

['00207b6d-3901-4f88-8fec-4ec9ce2e4019', '00277f20-a6f3-40fc-b582-33795727a3f9', '0043f2aa-7a23-4a9a-a83b-9237a365fc88', '008c7f1a-a6ee-46f4-97ac-cbac4eacc5b3']


In [108]:
seed = 6

driams_dataset = "B"
n_test_samples = int(0.2*len(samples_list))




['88d377e7-7218-4d10-8234-71f121093787',
 '0e8ce4e7-4848-4360-8118-aa7273d7f662',
 'cb8434fd-96c6-4b90-b5a4-635f919cf9a5',
 'f0efaf69-1f2f-4b4c-9e64-9f28732fd09d']

In [94]:
test_spectra = test_df[["species", "sample_id"]].drop_duplicates(subset=["species", "sample_id"]).values.tolist()
prediction_data = []

for sp, sid in test_spectra:
    for dr in drugs_set:
        prediction_data.append([sp, sid, dr, driams_dataset])
        
predictions_df = pd.DataFrame(prediction_data)
predictions_df.columns = ["species", "sample_id", "drug", "dataset"]
predictions_df = pd.merge(predictions_df,test_df,on=["species", "sample_id", "drug", "dataset"], how='outer')
predictions_df = predictions_df.fillna(-1)
predictions_df.loc[:, "response"] = predictions_df["response"].astype(int)
predictions_df = predictions_df[test_df.columns]
predictions_df


<ipython-input-94-81856d2cb6fa>:12: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  predictions_df.loc[:, "response"] = predictions_df["response"].astype(int)


,species,sample_id,drug,response,dataset
0,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Amikacin,0,B
1,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Ampicillin,1,B
2,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Benzylpenicillin,-1,B
3,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Cefepime,0,B
4,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Cefoxitin,0,B
...,...,...,...,...,...
15703,Staphylococcus aureus,02cf7dfe-c0c0-46e4-8848-168ddb20a336,Rifampicin,0,B
15704,Staphylococcus aureus,02cf7dfe-c0c0-46e4-8848-168ddb20a336,Teicoplanin,0,B
15705,Staphylococcus aureus,02cf7dfe-c0c0-46e4-8848-168ddb20a336,Tetracycline,0,B
15706,Staphylococcus aureus,02cf7dfe-c0c0-46e4-8848-168ddb20a336,Tigecycline,0,B


In [46]:
merged_df = pd.merge(df1,test_df,on=["species", "sample_id", "drug", "dataset"], how='outer')
merged_df

,species,sample_id,drug,dataset,response
0,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Amikacin,B,0.0
1,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Ampicillin,B,1.0
2,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Benzylpenicillin,B,NaN
3,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Cefepime,B,0.0
4,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Cefoxitin,B,0.0
...,...,...,...,...,...
15703,Staphylococcus aureus,6b826ffb-3ff9-4127-98e9-3f3dda58f82a,Rifampicin,B,0.0
15704,Staphylococcus aureus,6b826ffb-3ff9-4127-98e9-3f3dda58f82a,Teicoplanin,B,0.0
15705,Staphylococcus aureus,6b826ffb-3ff9-4127-98e9-3f3dda58f82a,Tetracycline,B,0.0
15706,Staphylococcus aureus,6b826ffb-3ff9-4127-98e9-3f3dda58f82a,Tigecycline,B,0.0


In [47]:
merged_df.fillna(-1)

,species,sample_id,drug,dataset,response
0,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Amikacin,B,0.0
1,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Ampicillin,B,1.0
2,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Benzylpenicillin,B,-1.0
3,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Cefepime,B,0.0
4,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Cefoxitin,B,0.0
...,...,...,...,...,...
15703,Staphylococcus aureus,6b826ffb-3ff9-4127-98e9-3f3dda58f82a,Rifampicin,B,0.0
15704,Staphylococcus aureus,6b826ffb-3ff9-4127-98e9-3f3dda58f82a,Teicoplanin,B,0.0
15705,Staphylococcus aureus,6b826ffb-3ff9-4127-98e9-3f3dda58f82a,Tetracycline,B,0.0
15706,Staphylococcus aureus,6b826ffb-3ff9-4127-98e9-3f3dda58f82a,Tigecycline,B,0.0


,species,sample_id,drug,response,dataset
0,Klebsiella pneumoniae,ca568529-351a-43af-8cec-7175488f66ea,Amikacin,0,B
1,Klebsiella pneumoniae,ca568529-351a-43af-8cec-7175488f66ea,Ampicillin,1,B
16,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Amikacin,0,B
17,Proteus mirabilis,0fd4fd1a-8233-400d-bdee-f0c13819dc6b,Ampicillin,1,B
32,Staphylococcus aureus,b1fa5e1c-d013-4716-b4b0-6552ad1c50de,Cefepime,0,B
...,...,...,...,...,...
32338,Stenotrophomonas maltophilia,fbbb0e92-577e-4cdd-a8ae-05367a477403,Cotrimoxazol,0,B
32339,Staphylococcus aureus,68d1147b-1606-46e3-a515-e992f0f84455,Cefepime,0,B
32344,Staphylococcus aureus,68d1147b-1606-46e3-a515-e992f0f84455,Benzylpenicillin,1,B
32358,Staphylococcus aureus,57043b13-3ba8-4f30-83ac-2416c23cec3a,Cefepime,0,B


['ca568529-351a-43af-8cec-7175488f66ea',
 '47a2835e-95dc-40d3-8ef4-a6054940b0c8',
 '49963f5a-7d57-42ab-b91b-e10825987113',
 '99706559-2e3a-4a6a-a579-3f2b5fdee5be',
 '5f4263b3-cf7f-45be-b910-080fd4e4f188',
 '667c6857-01b0-4710-b024-af296e45825e',
 'b374dfd6-6a68-4651-94b1-5dbe55b2d413',
 '523e3156-24dd-42a7-a19a-52a549892426',
 'e26e6c65-428c-4112-9136-365d91a27800',
 'dc2383e3-c96f-4e25-a9ed-790f73909a97',
 'eed06320-c82a-43a2-ad35-139e4e082044',
 'bf39f9b5-3757-4aac-a0dc-607d95008f87',
 'e0d3fda1-cb6a-4fc4-b654-37904964db3b',
 '9d18bf07-46e5-4f4e-97fa-546f6b9dd589',
 '6fa132a4-344e-4201-920e-bf819dd3785a',
 'dda60890-04cf-4e4c-a5ad-809b1fc661be',
 '3453f792-42ac-4881-acb5-7847c2cc940d',
 '8ee4f997-7eae-4a95-9d21-e4942905c8f9',
 '479eaa82-f64f-4c04-9045-836b467de090',
 '0fd4d1eb-54e0-46d2-9dcc-ac8e044a93ff',
 'b313b3f2-df8d-4619-9609-7369dfb5c178',
 '6a522e8d-16ad-41e6-b335-d49b97cb385a',
 '0b398b94-62b7-4e44-8e1c-deba13157f86',
 '1e819f82-6766-485b-b150-0912219e5c8b',
 'b8eb58cf-9f30-

In [5]:
for dataset in ["A", "B", "C", "D"]:
    df = driams_long_table[driams_long_table["dataset"]==dataset]
    print(len(df["drug"].unique()))



57
33
44
32
